#### Stream Reddit data into Google Cloud Platform

In [1]:
# import dependent libraries
import datetime
import praw
import pandas as pd
import json
import os
from google.cloud import storage
import time

with open('config.text') as json_file:  
    data = json.load(json_file)

# reddit API connection
reddit = praw.Reddit(client_id=data['client_id'],
                     client_secret=data['client_secret'],
                     user_agent=data['user_agent'],
                     username=data['username'],
                     password=data['password'])

def upload_blob(bucket_name, source_file_name, destination_blob_name):
    """Uploads a file to the bucket."""
    storage_client = storage.Client()
    bucket = storage_client.get_bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)

    blob.upload_from_filename(source_file_name)

    print('File {} uploaded to {}.'.format(
        source_file_name,
        destination_blob_name))

#### Obtain a Subreddit

To obtain a Subreddit instance, pass the subreddit’s name when calling subreddit on your Reddit instance.

In [2]:
# assume you have a Reddit instance bound to variable `reddit`
subreddit = reddit.subreddit('datascience')

print(subreddit.display_name)
print(subreddit.title) 

datascience
Data Science


In [3]:
topics_dict = {'submission_id':[],
               'user_id':[],
               'submission_timestamp':[],
               'title':[],
               'body':[]}
count = 0
for submission in subreddit.stream.submissions():
    topics_dict['submission_id'].append(submission.id)
    topics_dict['user_id'].append(submission.author)
    topics_dict['submission_timestamp'].append(submission.created_utc)
    topics_dict['title'].append(submission.title.encode('utf-8').strip())
    topics_dict['body'].append(submission.selftext.encode('utf-8').strip())
    print 'post detected'
    print 'current counter: ' + str(count)
    if count == 50:
        pd.DataFrame(topics_dict).to_csv('./temp_blob.csv', index=False, doublequote=True, sep='|')
        upload_blob('raw_dataset', 'temp_blob.csv', 
                    'reddit/raw_posts/reddit_datascience_post_stream_' + str(int(time.time())) + '.csv')
        count = 0

    count += 1

post detected
current counter: 0
post detected
current counter: 1
post detected
current counter: 2
post detected
current counter: 3
post detected
current counter: 4
post detected
current counter: 5
post detected
current counter: 6
post detected
current counter: 7
post detected
current counter: 8
post detected
current counter: 9
post detected
current counter: 10
post detected
current counter: 11
post detected
current counter: 12
post detected
current counter: 13
post detected
current counter: 14
post detected
current counter: 15
post detected
current counter: 16
post detected
current counter: 17
post detected
current counter: 18
post detected
current counter: 19
post detected
current counter: 20
post detected
current counter: 21
post detected
current counter: 22
post detected
current counter: 23
post detected
current counter: 24
post detected
current counter: 25
post detected
current counter: 26
post detected
current counter: 27
post detected
current counter: 28
post detected
current co

KeyboardInterrupt: 